##### Copyright 2019 Google LLC.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Closed Form Matting Energy
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/matting.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/matting.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


In image matting the input image $I$ is assumed to be a linear combination of a foreground image $F$ and a background image $B$. For a pixel $j$ of $I$, the color of the pixel can therefore be expressed as $I_j = \alpha_j F_j +(1−\alpha_j)B_j$,
where $\alpha_j$ is the foreground opacity for the pixel $j$. The opacity image made of all the $\alpha_j$ pixels is called a matte. 
<center>
<img src="https://github.com/frcs/alternative-matting-laplacian/raw/master/GT04.png" width="283" height="200" />
<img src="https://github.com/frcs/alternative-matting-laplacian/raw/master/alpha0-GT04.png" width="283" height="200" />
</center>

Using a trimap (white for foreground, black for background, and gray for unknown pixels),
<center>
<img src="https://github.com/frcs/alternative-matting-laplacian/raw/master/trimap-GT04.png" width="283" height="200" />
</center>

an optimization can be formulated to retrieve the unknow pixel opacities. 

This colab demonstrates how to use the image matting loss implemented in TensorFlow Graphics to perform this optimization. This loss is derived from the paper untitled "A Closed Form Solution to Natural Image Matting" from Levin et al. The loss was "tensorized" inspired by "Deep-Energy: Unsupervised Training of Deep Neural Networks" from Golts et al.




## Setup & Imports
If Tensorflow Graphics is not installed on your system, the following cell can install the Tensorflow Graphics package for you.

In [0]:
#!pip install tensorflow_graphics
!git clone https://github.com/tensorflow/graphics.git
!mv graphics/tensorflow_graphics tensorflow_graphics
!rm -rf graphics

Now that Tensorflow Graphics is installed, let's import everything needed to run the demos contained in this notebook.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow_graphics.image import matting

tf.enable_eager_execution()

## Imports the image and trimap.
Download the image and trimap from alphamatting.com.

In [0]:
# Downloads dataset from alphamatting.com
!wget http://www.alphamatting.com/datasets/zip/input_training_lowres.zip
!wget http://www.alphamatting.com/datasets/zip/trimap_training_lowres.zip
!wget http://www.alphamatting.com/datasets/zip/gt_training_lowres.zip

!unzip input_training_lowres.zip -d input_training_lowres
!unzip trimap_training_lowres.zip -d trimap_training_lowres
!unzip gt_training_lowres.zip -d gt_training_lowres

In [0]:
# Reads and decode images.
source = tf.read_file('input_training_lowres/GT04.png')
source = tf.cast(tf.io.decode_png(source), tf.float64) / 255.0
source = tf.expand_dims(source, axis=0)
trimap = tf.read_file('trimap_training_lowres/Trimap1/GT04.png')
trimap = tf.cast(tf.io.decode_png(trimap), tf.float64) / 255.0
trimap = tf.reduce_mean(trimap, axis=-1, keepdims=True)
trimap = tf.expand_dims(trimap, axis=0)
gt_matte = tf.read_file('gt_training_lowres/GT04.png')
gt_matte = tf.cast(tf.io.decode_png(gt_matte), tf.float64) / 255.0
gt_matte = tf.reduce_mean(gt_matte, axis=-1, keepdims=True)
gt_matte = tf.expand_dims(gt_matte, axis=0)

# Resize images to improve performance.
source = tf.image.resize(
    source,
    tf.shape(source)[1:3] // 2,
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
trimap = tf.image.resize(
    trimap,
    tf.shape(trimap)[1:3] // 2,
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
gt_matte = tf.image.resize(
    gt_matte,
    tf.shape(gt_matte)[1:3] // 2,
    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

# Shows images
fig = plt.figure(figsize=(22, 18))
axes = fig.add_subplot(1, 3, 1)
axes.grid(False)
axes.set_title('Input image', fontsize=14)
plt.imshow(source[0, ...].numpy())
axes = fig.add_subplot(1, 3, 2)
axes.grid(False)
axes.set_title('Input trimap', fontsize=14)
plt.imshow(trimap[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)
axes = fig.add_subplot(1, 3, 3)
axes.grid(False)
axes.set_title('GT matte', fontsize=14)
plt.imshow(gt_matte[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)

## Extract the foreground and background constraints from the trimap image.

In [0]:
# Extracts the foreground and background constraints from the trimap image.
foreground = tf.cast(tf.equal(trimap, 1.0), tf.float64)
background = tf.cast(tf.equal(trimap, 0.0), tf.float64)

# Shows foreground and background constraints.
fig = plt.figure(figsize=(22, 18))
axes = fig.add_subplot(1, 2, 1)
axes.grid(False)
axes.set_title('Foreground constraints', fontsize=14)
plt.imshow(foreground[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)
axes = fig.add_subplot(1, 2, 2)
axes.grid(False)
axes.set_title('Background constraints', fontsize=14)
plt.imshow(background[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)

## Setup & run the optimization

Setup the matting loss function using TensorFlow Graphics.

In [0]:
# Initializes the matte with random values.
matte_shape = tf.concat((tf.shape(source)[:-1], (1,)), axis=-1)
matte = tf.Variable(
    tf.random.uniform(
        shape=matte_shape, minval=0.0, maxval=1.0, dtype=tf.float64))
# Creates the closed form matting Laplacian
L, _ = matting.build_matrices(source)


# Function computing the loss and applying the gradient.
@tf.function
def optimize(optimizer):
  with tf.GradientTape() as tape:
    tape.watch(matte)
    # Computes a loss enforcing the trimap constraints.
    constraints = tf.reduce_mean((foreground + background) *
                                 tf.math.squared_difference(matte, foreground))
    # Computes the matting loss.
    smoothness = matting.loss(matte, L)
    # Sums up the constraint and matting losses.
    total_loss = 100 * constraints + smoothness
  # Computes and applies the gradient to the matte.
  g = tape.gradient(total_loss, [matte])
  optimizer.apply_gradients(zip(g, (matte,)))

Run the Adam optimizer for 400 iterations. 

In [0]:
# Runs the Adam optimizer for 400 iterations.
optimizer = tf.train.AdamOptimizer(learning_rate=1.0)
nb_iterations = 400
for it in range(nb_iterations):
  optimize(optimizer)

# Displays the result.
fig = plt.figure(figsize=(22, 18))
axes = fig.add_subplot(1, 4, 1)
axes.grid(False)
axes.set_title('Input image', fontsize=14)
plt.imshow(source[0, ...].numpy())
axes = fig.add_subplot(1, 4, 2)
axes.grid(False)
axes.set_title('Input trimap', fontsize=14)
plt.imshow(trimap[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)
axes = fig.add_subplot(1, 4, 3)
axes.grid(False)
axes.set_title('Matte', fontsize=14)
plt.imshow(matte[0, ..., 0].numpy(), cmap='gray', vmin=0, vmax=1)
axes = fig.add_subplot(1, 4, 4)
axes.grid(False)
axes.set_title('Input image x Matte', fontsize=14)
plt.imshow((source * matte)[0, ...].numpy())